In [1]:
print("ok")

ok


In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\Asus\\Desktop\\medibot\\End-to-end-Medical-Chatbot-Generative-AI'

In [3]:
# %pip install -U langchain-community


In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\Asus\anaconda3\envs\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    
    return documents

In [6]:
# %pip install pypdf

In [7]:
# %pip uninstall PIL
# %pip install Pillow




In [8]:
extracted_data = load_pdf_file(data="Data/")

In [9]:
# extracted_data

In [10]:
#split the data into text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(extracted_data)
    
    return texts_chunks

In [11]:
text_chunks= text_split(extracted_data) 
print("length of text chunks: ",{len(text_chunks)})

length of text chunks:  {5859}


In [12]:
# text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_hugging_face_embaddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
# %pip install sentence-transformers

In [16]:
embeddings = download_hugging_face_embaddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_22008\4000906065.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_results = embeddings.embed_query("hello world")
print("Length ", len(query_results))

Length  384


In [18]:
query_results

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [19]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
# %pip install google-generativeai


In [22]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [23]:
# %pip install google
# %pip install google-api-python-client
# %pip install grpcio



In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY, environment="us-west1-gcp")
index_name = "medicalbot"

# pc.create_index(name=index_name,
#                 dimension=384, 
#                 metric="cosine", 
#                 spec=ServerlessSpec(
#                     cloud=  "aws",
#                     region= "us-east-1",
#                     )

#                 )

In [25]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [26]:
%pip install langchain_pinecone

Note: you may need to restart the kernel to use updated packages.


In [27]:
# from langchain_pinecone import PineconeVectorStore

#     # embed eachchunk and upsert the embendings into your pincone index
# docsearch = PineconeVectorStore.from_documents(
#     documents =text_chunks, 
#     index_name=index_name,
#     embedding=embeddings, 
#     )

In [28]:
#load Existing index 
from langchain_pinecone import PineconeVectorStore
# embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings, 
    )

c:\Users\Asus\anaconda3\envs\medibot\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [31]:
retriever_docs=retriever.invoke("what is HIV?")

In [32]:
retriever_docs

[Document(id='90cc54f3-fdd1-4c30-9846-17be8145d3d1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 97.0, 'page_label': '98', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'total_pages': 637.0}, page_content='can be measured by a special instrument.\nHuman immunodeficiency virus (HIV)—A transmis-\nsible retrovirus that causes AIDS in humans. Two\nforms of HIV are now recognized: HIV-1, which caus-\nes most cases of AIDS in Europe, North and South\nAmerica, and most parts of Africa; and HIV-2, which\nis chiefly found in West African patients. HIV-2, dis-\ncovered in 1986, appears to be less virulent than HIV-\n1, but may also have a longer latency period.\nImmunofluorescent assay (IFA) —A blood test'),
 Document(id='a3b00477-6e4e-4743-875d-a4d630e9371f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 97.0, '

In [33]:
import google.generativeai as genai


# Configure Gemini API Key
genai.configure(api_key=GEMINI_API_KEY)

# Create the model
llm = genai.GenerativeModel(
    "gemini-1.5-flash",              # Use the specific Gemini model
    generation_config={
        "temperature": 0.4,
        "max_output_tokens": 500
    }
)

# Now you can generate responses!



In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
# %pip install langchain-google-genai


Note: you may need to restart the kernel to use updated packages.


In [41]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-pro",
 # or "gemini-1.5-pro", check your available models
    google_api_key=GEMINI_API_KEY  # Put your actual API key here
)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "you are an assistant for question-answering task "
    "use the following pieces of retriveed context to answer "
    "the question. if you don't know the answer, just say that "
    "you don't know, use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)




In [57]:
# Use the wrapped LangChain LLM (`llm`) INSTEAD of your raw model
question_answer_chain = create_stuff_documents_chain(llm, prompt)
reg_chain = create_retrieval_chain(retriever, question_answer_chain)


In [59]:
# For direct completion, use LangChain's LLM methods, not the raw Google API
response = reg_chain.invoke({"input":"what is lahore?"})   # Use .invoke()
print(response["answer"])   # Use .content for the result


I don't know. The provided text does not contain any information about Lahore. The context discusses medical terms such as anal fissures, polyps, and persistent generalized lymphadenopathy.
